For the preprocessing of the data, my main objective was to simplify the input and change the labels to match indices of output predictions (0-8) for each category of possible product. Since I had noticed, by printing some of the input image tensors, that all the points were either 0 or 255, I had thuis simplified the input images into 1 or 0 input. Beyond that I had also used unsqueez on the input to make the input chanels match the right dimensions for the later training of the model. I had also originally transfered my tensors to the gpu but had memory errors later on and had to revert back. (This error had caused my late submission to the leaderboard)

In [4]:
# your code here
import numpy as np
import torch
import torch.utils.data.dataloader
import torch.utils.data.dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import gc
import os

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

#Since only black and white (not grayscale) turn all non 0 to 1 for easier computation
torch.cuda.empty_cache()
gc.collect()

z = np.where(x>0,1,x)

v = np.where(y==1,0,y)
v = np.where(v==2,1,v)
v = np.where(v==3,2,v)
v = np.where(v==4,3,v)
v = np.where(v==6,4,v)
v = np.where(v==8,5,v)
v = np.where(v==9,6,v)
v = np.where(v==12,7,v)
v = np.where(v==16,8,v)

x_tr = torch.from_numpy(z)
y_tr = torch.from_numpy(v)

x_tr = x_tr.to(torch.float)
y_tr = y_tr.to(torch.long)


print(x_tr.size())
print(y_tr.size())

# indeces = torch.randperm(x_tr.size()[0])
# x_tr = x_tr[indeces]
# y_tr = y_tr[indeces]

x_tr = x_tr.unsqueeze(1)

print(torch.cuda.is_available())

x_tr = x_tr
y_tr = y_tr

print(x_tr.device)
print(y_tr.device)

torch.Size([11000, 64, 64])
torch.Size([11000])
True
cpu
cpu


For the architecture of my model, I had taken inspiration from both AlexNet and VGG to create a sort of combination of the two. I had experimented with different number of layers but found the best results to be with VGG type filters for the convolutional layers, a constant 3x3 with padding 1, non-changing output size within each convolutional layer similar to AlexNet, and a final choice of 3 Linear layers with descending output sizes. I also added 2 layers of dropout since I had initially noticed that my model was overfitting the data. I had added these after initially adding L2 regularization, since by itself the regularization was either too much or not enough regularization. For the optimization step, I had first chosen to do batch sizes of 50 since it was initially close to the limit of what my gpu could handle, and with higher batch sizes models produce better gradient estimates, thus having a better prediction. I had also shuffled the data before training as a general rule of thumb. After some initial research I had found that most, and the best, CNNs used cross entropy loss as the loss function so I had chosen to do the same. Finally for the actual optimizer, the choice of hyper-parameters was from experimenting, which was initally easy on the gpu. I found that with a learning rate of 0.001, the model was optimal. Although a lower learning rate and a higher number of epochs would eventually get me closer to the optimum, this took too long to train, and with a learning rate that was higher my model was overshooting the optimum and combined with momentum even started moving away from the optimum. The weight_decay hyper-parameter was used for L2 regularization and I simply chose the value through experimenting. Beyond the actual implementation,, I had also tried to add learning rate decay but found that training then became very slow so opted out from it.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,64,3,padding=1)
        self.conv2 = nn.Conv2d(64,128,3,padding=1)
        
        self.conv3 = nn.Conv2d(128,256,3,padding=1)
        self.conv4 = nn.Conv2d(256,256,3,padding=1)
        #14

        # self.conv5 = nn.Conv2d(64,128,3,padding=1)
        # self.conv6 = nn.Conv2d(128,128,3,padding=1)
        # self.conv7 = nn.Conv2d(128,128,3,padding=1)

        self.drop1 = nn.Dropout(0.5)
        self.drop2 = nn.Dropout(0.3)
        self.f1 = nn.Linear(16384,8192)
        self.f2 = nn.Linear(8192,4096)
        self.f3 = nn.Linear(4096,9)
        self.soft = nn.Softmax()
        # self.f4 = nn.Linear(380,9)
        # self.f5 = nn.Linear(120,9)
        # self.f6 = nn.Linear(100,9)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)),2)
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        #x = self.drop1(x)

        x = F.relu(self.conv3(x))
        x = F.max_pool2d(F.relu(self.conv4(x)),2)

        # x = F.relu(self.conv5(x))
        # x = F.relu(self.conv6(x))
        # x = F.max_pool2d(F.relu(self.conv7(x)),2)

        # x = F.relu(self.conv8(x))
        # x = F.relu(self.conv9(x))
        # x = F.max_pool2d(F.relu(self.conv10(x)),2)

        x = torch.flatten(x,1)
        x = self.drop1(x)
        x = F.relu(self.f1(x))
        x = self.drop2(x)
        x = F.relu(self.f2(x))
        # x = F.relu(self.f3(x))
        # x = F.relu(self.f4(x))
        # x = F.relu(self.f5(x))
        x = self.f3(x)
        return x



device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net = Net()

data = torch.utils.data.TensorDataset(x_tr,y_tr)
batch = 50

dataloader = torch.utils.data.DataLoader(data, batch,True)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001,momentum=0.9, weight_decay=0.005)
##optimizer = optim.Adam(net.parameters(),lr=0.001,betas=(0.9,0.999),weight_decay=0.005)
scheduler = optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.01,total_iters=10)

prior_loss = 2000
for epoch in range(100):

    
    running_accuracy = 0.0
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data

        optimizer.zero_grad()
        output = net(inputs)

        correct = torch.sum(labels == torch.argmax(output, dim=1)).item()
        running_accuracy += correct / batch

        loss = criterion(output,labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(running_loss)
    if(prior_loss - running_loss < 0.05): break
    prior_loss = running_loss 
      
    

478.5147457122803
470.3300688266754
469.7131927013397
469.22398042678833
468.56616854667664
467.03728556632996
461.92041981220245
426.99839997291565
363.41094648838043
336.8827098608017
319.77425813674927
299.24364817142487
281.4598785638809
262.47520637512207
245.47710764408112
231.4004549384117
217.5665807723999
205.7385209798813
196.27826726436615
186.44239872694016
177.18075597286224
168.48542866110802
156.2542008459568
150.64380049705505
141.8523726463318
132.85521295666695
125.4924908876419
122.7407845556736
120.09815749526024
111.03152757883072
103.53109304606915
101.5867197662592
99.68605595827103
90.78144611418247
86.97434209287167
80.50772165507078
79.69040144979954
72.72028091549873
68.64226211607456
64.77623284608126
59.540625140070915
56.92116494476795
55.476712457835674
52.21532768011093
47.429237212985754
46.29065925627947
42.00908836722374
38.04845245182514
35.445669235661626
31.523236643522978
31.34634944051504
28.901652980595827
26.173120699822903
23.454110626131296
2

In [ ]:
correct = 0
total = 0

z = np.where(x_test_half>0,1,0)

y_tt = np.where(y_test_half==1,0,y_test_half)
y_tt = np.where(y_tt==2,1,y_tt)
y_tt = np.where(y_tt==3,2,y_tt)
y_tt = np.where(y_tt==4,3,y_tt)
y_tt = np.where(y_tt==6,4,y_tt)
y_tt = np.where(y_tt==8,5,y_tt)
y_tt = np.where(y_tt==9,6,y_tt)
y_tt = np.where(y_tt==12,7,y_tt)
y_tt = np.where(y_tt==16,8,y_tt)

x_test_half = torch.from_numpy(z)
y_test_half = torch.from_numpy(y_tt)

print(x_test_half.size())
print(y_test_half.size())

x_test_half = x_test_half.to(torch.float)

x_test_half = x_test_half.unsqueeze(1)

test_data = torch.utils.data.TensorDataset(x_test_half,y_test_half)
testloader = torch.utils.data.DataLoader(test_data)

with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        outputs = net(inputs)
        # print(torch.argmax(outputs,dim=1))
        # print(labels.item())
        correct += torch.sum(labels == torch.argmax(outputs, dim=1)).item()
        total += labels.size(0)

test_accuracy = 100 * (correct/total)
print("Test accuracy: {} %".format(test_accuracy))

torch.Size([4500, 64, 64])
torch.Size([4500, 1])
Test accuracy: 83.17777777777778 %
